# Reconciliation Monthly Stats

We need to load in these libraries into our notebook in order to query, load, manipulate and view the data

In [ ]:
import base64
from config import Config

import os

import requests
from IPython import get_ipython
from IPython.display import display, Markdown

%load_ext sql
%config SqlMagic.displaylimit = 5
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

# Parameters cell for external parameters via papermill (job running this notebook will insert a parameter cell below this). This cell has a tag of with the name "parameters" that is used by papermill

e.g.
param1 = "some_value"

This will create the connection to the database and prep the jupyter magic for SQL

In [ ]:
%sql $Config.SQLALCHEMY_DATABASE_URI

Simplest query to run to ensure our libraries are loaded and our DB connection is working

In [ ]:
%%sql
set time zone 'UTC';

In [ ]:
from datetime import datetime, timezone, timedelta
from dateutil.relativedelta import relativedelta
import pytz

def get_utc_timezone_adjusted_date(target_date) -> str:
    target_datetime = datetime.combine(target_date, datetime.min.time())
    hours = target_datetime.astimezone(pytz.timezone("America/Vancouver")).utcoffset().total_seconds() / 60 / 60
    target_date = target_datetime.replace(tzinfo=timezone.utc) + relativedelta(hours=-hours)
    return target_date.replace(tzinfo=None).strftime('%Y-%m-%d %H:%M:%S')

def get_first_last_month_dates_in_utc() -> tuple[str, str]:
  current_time = datetime.now(pytz.timezone("America/Vancouver"))
  last_month = current_time - relativedelta(months=1)
  from_date = last_month.replace(day=1)
  from_date = get_utc_timezone_adjusted_date(from_date)
  to_date = (last_month.replace(day=1) + relativedelta(months=1)) - timedelta(days=1)
  to_date = get_utc_timezone_adjusted_date(to_date)
  return from_date, to_date

from_date, to_date = get_first_last_month_dates_in_utc()
print(f'Using from_date: {from_date} and to_date: {to_date}')

Query ...

In [ ]:
%%sql monthly_reconciliation_summary  <<
SELECT
    id as transaction_id,
    (created_on AT TIME ZONE 'UTC' AT TIME ZONE 'America/Vancouver')::date AS created_date_pacific,
    total,
    service_fees,
    total - service_fees as subtotal,
    payment_method_code,
    corp_type_code,
    created_by,
    payment_date,
    refund_date,
    invoice_status_code
FROM
    invoices
WHERE
    corp_type_code = :partner_code
    AND total > 0
    AND invoice_status_code in ('PAID', 'CREDITED', 'REFUNDED', 'CANCELLED', 'REFUND_REQUESTED')
    AND payment_method_code in ('PAD','EJV') 
    AND ((created_on >= :from_date and created_on <= :to_date) 
         OR (refund_date >= :from_date and refund_date <= :to_date))
ORDER BY
    1;

Save to CSV

In [ ]:
filename_summary = os.path.join(os.getcwd(), r'data/')+partner_code+'_monthly_reconciliation_summary_' + datetime.strftime(datetime.now()-timedelta(1), '%Y-%m') +'.csv'
df_summary = monthly_reconciliation_summary.DataFrame()
with open(filename_summary, 'w') as f:
    f.write('Monthly Reconciliation Summary:\n\n')
    if df_summary.empty:
        f.write('No Data Retrieved')
    else:
        df_summary.to_csv(f, sep=',', encoding='utf-8', index=False)

disbursement summary

In [ ]:
# Define the partner codes for which the disbursement summary should be executed
partners_for_disbursement_summary = ['CSO', 'VS']

print(f"Processing for partner_code: {partner_code}")
print(f"Partners for disbursement summary: {partners_for_disbursement_summary}")

# Only date part, no time saved for this field, also I believe this is pacific, it comes straight
# from the feedback files.
from_date = from_date.split(' ')[0] 
to_date = to_date.split(' ')[0]
print(f'Using from_date: {from_date} and to_date: {to_date}')
if partner_code in partners_for_disbursement_summary:
    print(f"Partner code {partner_code} found in the list, executing SQL query.")
    query = f"""
    SELECT id as transaction_id, disbursement_date::date as disbursement_date_pacific, disbursement_reversal_date::date disbursement_reversal_date_pacific, total, service_fees, total - service_fees as subtotal, payment_method_code, corp_type_code, created_by, disbursement_status_code
    FROM invoices
    WHERE corp_type_code = :partner_code
    AND invoice_status_code = 'PAID'
    AND payment_method_code in ('PAD','EJV')
    AND ((disbursement_date >= :from_date AND disbursement_date <= :to_date)
        OR (disbursement_reversal_date >= :from_date AND disbursement_reversal_date <= :to_date))
    order by 1;
    """

    display(Markdown(f"## Running query for partner: {partner_code}"))
    results = get_ipython().run_cell_magic('sql', '', query)
    monthly_reconciliation_disbursed = results.DataFrame()  # Convert the results to a DataFrame for later use
else:
    print(f"Partner code {partner_code} not in the list, skipping SQL query.")
    monthly_reconciliation_disbursed = None

Save to another CSV

In [ ]:
if monthly_reconciliation_disbursed is not None and not monthly_reconciliation_disbursed.empty:
    filename_disbursed = os.path.join(os.getcwd(), 'data', f'{partner_code}_monthly_reconciliation_disbursed_' + datetime.strftime(datetime.now() - timedelta(1), '%Y-%m') + '.csv')
    print(f"Saving CSV for partner_code: {partner_code} at {filename_disbursed}")
    with open(filename_disbursed, 'w') as f:
        f.write('Monthly Reconciliation Disbursed:\n\n')
        if monthly_reconciliation_disbursed.empty:
            f.write('No Data Retrieved')
        else:
            monthly_reconciliation_disbursed.to_csv(f, sep=',', encoding='utf-8', index=False)
else:
    print(f"Partner code {partner_code} not in the list or no data to save, skipping CSV save.")


Authenticate

In [ ]:
payload = "grant_type=client_credentials"
basic_hash = base64.b64encode(f"{Config.NOTEBOOK_SERVICE_ACCOUNT_ID}:{Config.NOTEBOOK_SERVICE_ACCOUNT_SECRET}".encode())

headers = {
  'Content-Type': 'application/x-www-form-urlencoded',
  'Authorization': f'Basic {basic_hash.decode()}'
}
response = requests.request("POST", f"{Config.JWT_OIDC_ISSUER}/protocol/openid-connect/token", headers=headers, data=payload)

assert response.status_code == 200
notebook_service_account_token = response.json().get('access_token')

In [ ]:

partner_details = {
    'CSO': {
        "companyName": "Ministry of Justice",
        "addressLine1": "PO Box 9249, Stn Prov Govt",
        "addressLine2": "6th Floor, 850 Burdett Avenue",
        "city": "VICTORIA",
        "province": "BC",
        "areaCode": "V8W 9J2"
    },
    'VS': {
        "companyName": "Vital Statistics Agency",
        "addressLine1": "PO Box 9657, Stn Prov Govt",
        "addressLine2": "",
        "city": "VICTORIA",
        "province": "BC",
        "areaCode": "V8W 9P3"
    },
    'RPT': {
        "companyName": "Property Taxation Branch",
        "addressLine1": "Ministry of Provincial Revenue",
        "addressLine2": "4th Floor, 1802 Douglas Street",
        "city": "VICTORIA",
        "province": "BC",
        "areaCode": "V8T 4K6"
    },
    'ESRA': {
        "companyName": "Ministry of Environment",
        "addressLine1": "Environmental Emergencies and Land Remediation",
        "addressLine2": "2nd Floor 10470-152nd Street",
        "city": "SURREY",
        "province": "BC",
        "areaCode": "V3R 0Y3"
    }

}

def generate_report(partner_code):
    API_BASE_URL = Config.REPORT_API_URL + '/reports'
    if not Config.REPORT_API_URL:
        raise ValueError("The REPORT_API_URL environment variable is not set or is empty")

    url = API_BASE_URL
    headers = {
        'Authorization': f'Bearer {notebook_service_account_token}',
        'Content-Type': 'application/json',
        'Accept': 'application/pdf'
    }

    query = """
    SELECT
        COUNT(*) AS transaction_count,
        SUM(total) AS total,
        TO_CHAR(DATE_TRUNC('month',current_date) - INTERVAL '1 month','Month') as month,
        corp_type_code
    FROM
        invoices
    WHERE
        corp_type_code = :partner_code
        AND invoice_status_code = 'PAID'
        AND payment_method_code IN ('PAD', 'EJV')
        AND DATE_TRUNC('month', created_on AT TIME ZONE 'UTC' AT TIME ZONE 'America/Vancouver') = DATE_TRUNC('month', current_date - INTERVAL '1 month')
    GROUP BY
        corp_type_code
    ORDER BY
        month;
    """

    result = %sql $query

    print(result)

    if result:
        df = result.DataFrame()

        # Rename columns to match the expected names in the template
        df.rename(columns={
            'corp_type_code': 'registry',
            'transaction_count': 'transCounts',
            'total': 'amount'
        }, inplace=True)

        df['symbol'] = '$'

        df['amount'] = df['amount'].apply(lambda x: f"{x:,.2f}")
        df['transCounts'] = df['transCounts'].apply(lambda x: f"{x:,.0f}")

        # Convert DataFrame to JSON-compatible format
        tableRows = df.to_dict(orient='records')

        current_date = datetime.now(tz=timezone.utc).strftime("%B %d, %Y")

        details = partner_details.get(partner_code, {})
        if not details:
            raise ValueError(f"No details found for partner code: {partner_code}")

        # Define the request body
        data = {
            "templateVars": {
                "date": current_date,
                "companyName": details["companyName"],
                "addressLine1": details["addressLine1"],
                "addressLine2": details["addressLine2"],
                "city": details["city"],
                "province": details["province"],
                "areaCode": details["areaCode"],
                "firstName": partner_code,
                "enterMonth": df['month'][0] if not df.empty else "N/A",
                "tableRows": tableRows
            },
            "templateName": "revenue_letter",
            "reportName": "revenue_letter"
        }

        response = requests.post(url, headers=headers, json=data)

        if response.status_code == 200:
            pdf_content = response.content

            pdf_filename = os.path.join(os.getcwd(), 'data', f'{partner_code}_revenue_letter.pdf')
            with open(pdf_filename, 'wb') as pdf_file:
                pdf_file.write(pdf_content)

            print("PDF report saved successfully as 'payment_receipt.pdf'")
        else:
            print('Failed to get the report:', response.text)
    else:
        print('No results returned from the SQL query')
generate_report(partner_code)